In [ ]:
import ur_analytic_ik
from ur_analytic_ik import ur5e
import numpy as np
np.set_printoptions(precision=4, suppress=True) # Prettier printing

In [ ]:
eef_pose0 = ur5e.forward_kinematics(0, 0, 0, 0, 0, 0)
eef_pose0


In [ ]:
theta45 = np.deg2rad(45)
eef_pose45 = ur5e.forward_kinematics(theta45, theta45, theta45, theta45, theta45, theta45)
eef_pose45

In [ ]:
%%timeit
ur5e.forward_kinematics(theta45, theta45, theta45, theta45, theta45, theta45)

In [ ]:
eef_pose = np.array(eef_pose0)
print(type(eef_pose), type(eef_pose0))
print(eef_pose.shape, eef_pose0.shape)
print(eef_pose.dtype, eef_pose0.dtype)

ur_analytic_ik.inspect(eef_pose) # works
# ur_analytic_ik.inspect(eef_pose0) # error

In [ ]:
eef_pose = np.array(eef_pose0)
print("All joints should be:", 0)
solutions = ur5e.inverse_kinematics(eef_pose)
solutions

In [ ]:
def filter_solutions(solutions):
    filtered = [solution for solution in solutions if not np.isnan(solution).any()]
    return filtered

filter_solutions(solutions)

In [ ]:
unique_solutions = np.unique(filter_solutions(solutions), axis=0)
unique_solutions

In [ ]:
%%timeit
np.unique(filter_solutions(solutions), axis=0)

In [ ]:
%%timeit
filter_solutions(solutions)

In [ ]:
def filter_solutions2(solutions):
    filtered = [solution for solution in solutions if not np.isnan(np.sum(solution))]
    return filtered

filter_solutions2(solutions)

In [ ]:
%%timeit
filter_solutions2(solutions)

In [ ]:
solution8 = solutions[-1]
solution8

In [ ]:
solution8.shape

In [ ]:
np.rad2deg(solution8)

In [ ]:
ur5e.forward_kinematics(*solution8.T)

In [ ]:
eef_pose = np.array(eef_pose45)
print("All joints should be:", np.deg2rad(45))
ur5e.inverse_kinematics(eef_pose)

In [ ]:
%%timeit
ur5e.inverse_kinematics(eef_pose)

In [ ]:
ur5e.inverse_kinematics(eef_pose)

In [ ]:
ur5e.inverse_kinematics.__doc__